In [78]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


# Load data

In [87]:
import numpy as np
import pandas as pd
ysample = np.random.randint(2, size=(5000, 1))
xsample = np.random.randint(10, size=(5000, 1))
sampledata = pd.DataFrame(np.concatenate([ysample, xsample], axis=1), columns=['y', 'x'])

# Functions to compare timings with pure python versions with cython versions. 

In [112]:
def compare_time(current, reference, name):
    ratio = reference/current
    if ratio > 1:
        word = "faster"
    else:
        ratio = 1 / ratio 
        word = "slower"
        
    print("We are", "{0:.1f}".format(ratio), "times", word, "than the", name, "version.")

def print_report(computefunc):
#     assert np.all(computefunc(sampledata, 'y', 'x') == py_result)
    timeit_result = %timeit -o computefunc(sampledata, 'y', 'x')
    run_time = sum(timeit_result.all_runs) / timeit_result.repeat / timeit_result.loops
    compare_time(run_time, py_time, "pure Python")

# Pure python Version

In [81]:
def py_example(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    n = data.shape[0]
    for i in range(n):
      Xname, Yname = data.loc[i, x_name], data.loc[i, y_name]
      if Xname not in value_dict.keys():
          value_dict[Xname] = Yname
          count_dict[Xname] = 1
      else:
          value_dict[Xname] += Yname
          count_dict[Xname] += 1
    for i in range(n):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [82]:
py_result = py_example(sampledata, 'y', 'x')
py_result

array([0.50306748, 0.50306748, 0.49586777, ..., 0.48602151, 0.48602151,
       0.48289738])

In [83]:
timeit_result = %timeit -o py_example(sampledata, 'y', 'x')
py_time = sum(timeit_result.all_runs) / timeit_result.repeat / timeit_result.loops

109 ms ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
py_time

0.10945529428567428

# Pure python version compiled with Cython

In [9]:
%%cython -a
import numpy as np
def computefunc(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    n = data.shape[0]
    for i in range(n):
      Xname, Yname = data.loc[i, x_name], data.loc[i, y_name]
      if Xname not in value_dict.keys():
          value_dict[Xname] = Yname
          count_dict[Xname] = 1
      else:
          value_dict[Xname] += Yname
          count_dict[Xname] += 1
    for i in range(n):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [10]:
print_report(computefunc)

108 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
We are 1.0 times faster than the pure Python version.


# Adding types

In [105]:
%%cython -a
import numpy as np
cimport numpy as cnp

cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    cdef cnp.ndarray[double] result = np.zeros(data.shape[0])
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
    cdef Py_ssize_t n = data.shape[0]
    cdef Py_ssize_t Xname, Yname
    cdef unsigned int i
    for i in range(n):
      Xname = data[i, 1]
      Yname = data[i, 0]
      if Xname not in value_dict.keys():
          value_dict[Xname] = Yname
          count_dict[Xname] = 1
      else:
          value_dict[Xname] += Yname
          count_dict[Xname] += 1
    for i in range(n):
        Xname = data[i, 1]
        result[i] = (value_dict[Xname] - data[i, 0]) / (count_dict[Xname] - 1)
    return result

In [99]:
sampledata = np.concatenate([ysample, xsample], axis = 1)

In [106]:
print_report(computefunc)

652 µs ± 8.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
We are 167.8 times faster than the pure Python version.


# Efficient indexing with memoryviews

In [107]:
%%cython -a
import numpy as np
cimport numpy as cnp

cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    
    cdef cnp.ndarray[double] result = np.zeros(data.shape[0])
    cdef double[:] result_view = result
    
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
    
    cdef Py_ssize_t n = data.shape[0]
    cdef Py_ssize_t Xname, Yname
    
    cdef unsigned int i
    
    for i in range(n):
      Xname = data[i, 1]
      Yname = data[i, 0]
      if Xname not in value_dict.keys():
          value_dict[Xname] = Yname
          count_dict[Xname] = 1
      else:
          value_dict[Xname] += Yname
          count_dict[Xname] += 1
    for i in range(n):
        Xname = data[i, 1]
        result_view[i] = (value_dict[Xname] - data[i, 0]) / (count_dict[Xname] - 1)
    return result

In [108]:
print_report(computefunc)

641 µs ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
We are 170.7 times faster than the pure Python version.


# Improved efficient indexing with memoryviews when ndarrays are split

In [110]:
%%cython -a
import numpy as np
cimport numpy as cnp

cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    
    cdef int[:] x = data[:, 1].astype(np.intc)
    cdef int[:] y = data[:, 0].astype(np.intc)
    cdef int n = data.shape[0]
    
    cdef cnp.ndarray[double] result = np.zeros(n)
    cdef double[:] result_view = result
    
    cdef int[:] value_dict = np.zeros(n).astype(np.intc)
    cdef int[:] count_dict = np.zeros(n).astype(np.intc)
    
    cdef unsigned int i
    
    for i in range(n):
        value_dict[x[i]] += y[i]
        count_dict[x[i]] += 1
        
    for i in range(n):
        result_view[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
    return result

In [113]:
print_report(computefunc)

31.5 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
We are 3475.5 times faster than the pure Python version.


# Tuning indexing further

In [115]:
%%cython -a
import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    
    cdef int[:] x = data[:, 1].astype(np.intc)
    cdef int[:] y = data[:, 0].astype(np.intc)
    cdef int n = data.shape[0]
    
    cdef cnp.ndarray[double] result = np.zeros(n)
    cdef double[:] result_view = result
    
    cdef int[:] value_dict = np.zeros(n).astype(np.intc)
    cdef int[:] count_dict = np.zeros(n).astype(np.intc)
    
    cdef unsigned int i
    
    for i in range(n):
        value_dict[x[i]] += y[i]
        count_dict[x[i]] += 1
        
    for i in range(n):
        result_view[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
    return result

In [116]:
print_report(computefunc)

26.5 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
We are 4126.4 times faster than the pure Python version.


# Declaring the numpy array as contiguous

In [119]:
%%cython -a
import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    
    cdef int[::1] x = data[:, 1].astype(np.intc)
    cdef int[::1] y = data[:, 0].astype(np.intc)
    cdef int n = data.shape[0]
    
    cdef cnp.ndarray[double] result = np.zeros(n)
    cdef double[::1] result_view = result
    
    cdef int[::1] value_dict = np.zeros(n).astype(np.intc)
    cdef int[::1] count_dict = np.zeros(n).astype(np.intc)
    
    cdef unsigned int i
    
    for i in range(n):
        value_dict[x[i]] += y[i]
        count_dict[x[i]] += 1
        
    for i in range(n):
        result_view[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
    return result

In [120]:
print_report(computefunc)

24.7 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
We are 4434.4 times faster than the pure Python version.


# Using multiple threads

In [122]:
%%cython -a
import numpy as np
cimport numpy as cnp
cimport cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] computefunc(cnp.ndarray[long, ndim=2] data, y_name, x_name):
    
    cdef int[::1] x = data[:, 1].astype(np.intc)
    cdef int[::1] y = data[:, 0].astype(np.intc)
    cdef int n = data.shape[0]
    
    cdef cnp.ndarray[double] result = np.zeros(n)
    cdef double[::1] result_view = result
    
    cdef int[::1] value_dict = np.zeros(n).astype(np.intc)
    cdef int[::1] count_dict = np.zeros(n).astype(np.intc)
    
    cdef int i
    
    for i in prange(n, nogil = True):
        value_dict[x[i]] += y[i]
        count_dict[x[i]] += 1
        
    for i in prange(n, nogil = True):
        result_view[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
    return result

In [123]:
print_report(computefunc)

24.3 µs ± 683 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
We are 4510.4 times faster than the pure Python version.
